In [1]:
import torch
from torch import nn
from torch.nn import functional as F

### 1.自定义块

自定义块通常需要包含如下几个部分：

- **输入：** 输入数据作为其前向传播函数的参数
- **前向传播函数：** 输入通过前向传播函数的作用，产生输出
- **梯度：** 能够计算输出关于输入的梯度
- **模型参数：** 能够存储和访问前向传播计算所需的参数
- **初始化：** 能够对模型参数进行初始化

在构建自定义块时，一般会继承nn.Module类，然后修改__init__()和forward()函数

- \_\_init\_\_()函数中主要实现一些层和参数的初始化
- forward()函数中定义前向传播的实现过程

In [2]:
class MLP(nn.Module):
    def __init__(self) -> None:
        # 调用父类（nn.Module）的构造函数来执行必要的初始化
        super().__init__()
        self.hidden = nn.Linear(20, 256)    # 隐藏层
        self.out = nn.Linear(256, 10)       # 输出层
    
    # 定义模型的前向传播函数
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

net = MLP()
X = torch.rand(2, 20)
net(X)

tensor([[ 0.1647,  0.2726,  0.0836, -0.1052,  0.0880,  0.0883,  0.3569, -0.0391,
         -0.1113, -0.2461],
        [ 0.2843,  0.2576,  0.1843,  0.0019,  0.1960,  0.0899,  0.3481, -0.0906,
         -0.1912, -0.2069]], grad_fn=<AddmmBackward0>)

### 2.顺序块的实现

In [4]:
class MySequential(nn.Module):
    def __init__(self, *args) -> None:
        super().__init__()
        for idx, module in enumerate(args):
            # args为一个元组，按顺序存储了输入参数
            # _modules的类型是OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0931, -0.2852, -0.0623, -0.3300,  0.1176, -0.1481,  0.1376, -0.0824,
         -0.0252, -0.1004],
        [-0.1717, -0.2593, -0.0870, -0.1262, -0.0887,  0.0807,  0.1852, -0.0528,
          0.0004, -0.1297]], grad_fn=<AddmmBackward0>)

### 3.前向传播函数

我们可以在前向传播函数中自定义执行流程，引入Python的控制流，不参与梯度计算的固定参数等

In [5]:
class FixedHiddenMLP(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        # 固定权重参数，在训练过程中保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(-0.0522, grad_fn=<SumBackward0>)

### 4.自由组合各种块

In [7]:
class NestMLP(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
    
    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.0144, grad_fn=<SumBackward0>)